In [3]:
import pandas as pd

df_usagers = pd.read_csv("cleaned_files/usagers_clean.csv", sep=";", dtype=str)
df_vehicules= pd.read_csv("cleaned_files/véhicules_clean.csv", sep=";",dtype=str)
df_caracteristiques = pd.read_csv("cleaned_files/caractéristiques_clean.csv", sep=";", dtype=str)
df_lieux = pd.read_csv("csv/lieux_apres_pivot.csv", sep=";", dtype=str)

In [4]:

df = df_usagers.merge(df_vehicules,on=['Accident_Id','Numéro du véhicule'])

In [5]:
df.shape

(620105, 25)

ACCIDENTS AVEC DES VEHICULES DE MOBILITE DOUCE (EDPM / VELO / PIETONS)

In [6]:
catv = ["01 - Bicyclette", "80 - VAE", "50 - EDP à moteur", "60 - EDP sans moteur"]

# Filter DataFrame by list of user
filtered_df = df[df['Catégorie du véhicule'].isin(catv)]

s_acc = filtered_df["Accident_Id"]

list_acc = s_acc.to_list()

accidents_mobilites_douces = df[df['Accident_Id'].isin(list_acc)]
#print(filtered_acc_df)

In [7]:
df = accidents_mobilites_douces
df=df.sort_values(['Accident_Id','Catégorie du véhicule',"Catégorie usager"],ascending=True)

# Ajouter un index par groupe ID
df['row'] = df.groupby('Accident_Id').cumcount() + 1

# Transformer les lignes en colonnes
df_pivot_md = df.pivot(index='Accident_Id', columns='row', values=['Catégorie du véhicule','Catégorie usager', 'Gravité des blessures', 'Sexe', 'Année de naissance', 'Motif du déplacement', 'Equipement de sécurité 1'])

# Aplatir l'index des colonnes
df_pivot_md.columns = [f"{col[0]}{col[1]}" for col in df_pivot_md.columns]

# Réinitialiser l'index
df_pivot_md.reset_index(inplace=True)

JOINTURE AVEC LES AUTRES INFOS

In [8]:
accident = df_caracteristiques.merge(df_lieux,on = 'Accident_Id')
df_complete = df_pivot_md.merge(accident,on='Accident_Id')

A VOIR SI NECESSAIRE ? 

In [9]:
cols_a_supprimer = []

EXPORT EN CSV

In [10]:
df_complete.to_csv('datasets/table_complete_accidents_mobilité_douce_avec_pivot_véhicules.csv',sep=";",index=False)

ACCIDENTS QUI IMPLIQUENT DES PIETONS


In [5]:
df.shape

(619971, 48)

In [6]:
catu = ["3 - Piéton"]

# Filter DataFrame by list of user
filtered_df = df[df['catégorie_usager'].isin(catu)]

s_acc = filtered_df["Accident_Id"]

list_acc = s_acc.to_list()

accidents_piétons = df[df['Accident_Id'].isin(list_acc)]
#print(filtered_acc_df)

In [7]:
accidents_piétons.shape

(95822, 48)

In [8]:
accidents_piétons.to_csv("csv/accidents_piétons_sans_pivot.csv", index=False)

VELOS ET VAE (LES CYCLISTES, N'INCLUT PAS LES PIETONS HEURTES PAR DES VELOS)

In [9]:
catv = ["Vélo","Vélo électrique"]

# Filter DataFrame by list of user

filtered_df = df[df['Catégorie_véhicule'].isin(catv) & df['catégorie_usager'].isin(["1 - Conducteur"])]

s_acc = filtered_df["Accident_Id"]

list_acc = s_acc.to_list()

accidents_cyclistes = df[df['Accident_Id'].isin(list_acc)]
#print(filtered_acc_df)

In [10]:
accidents_cyclistes.shape

(53769, 48)

In [11]:
accidents_cyclistes.to_csv("csv/accidents_cyclistes_sans_pivot.csv", index=False)

ON GARDE UNE TABLE AVEC LES INFOS COMMUNES ET UNE AUTRE QU'ON VA PIVOTER


In [12]:
accidents_mobilites_douces_base = accidents_mobilites_douces.drop_duplicates('Accident_Id',keep='first').iloc[:,0:16]
accidents_mobilites_douces_vehicules = accidents_mobilites_douces.iloc[:,16:]

In [13]:
accidents_piétons_base = accidents_piétons.drop_duplicates('Accident_Id',keep='first').iloc[:,0:16]
accidents_piétons_vehicules = accidents_piétons.iloc[:,16:]

In [14]:
accidents_cyclistes_base = accidents_cyclistes.drop_duplicates('Accident_Id',keep='first').iloc[:,0:16]
accidents_cyclistes_vehicules = accidents_cyclistes.iloc[:,16:]

LE PIVOT DES VEHICULES POUR CHAQUE SOUS-GROUPE

In [15]:
df = accidents_mobilites_douces_vehicules
df=df.sort_values(['Num_Acc','Catégorie_véhicule',"catégorie_usager"],ascending=True)

# Ajouter un index par groupe ID
df['row'] = df.groupby('Num_Acc').cumcount() + 1

# Transformer les lignes en colonnes
df_pivot_md = df.pivot(index='Num_Acc', columns='row', values=['Catégorie_véhicule','catégorie_usager', 'gravité_blessure', 'sexe', 'année_naissance', 'motif_trajet', 'équipement_sécu1'])

# Aplatir l'index des colonnes
df_pivot_md.columns = [f"{col[0]}{col[1]}" for col in df_pivot_md.columns]

# Réinitialiser l'index
df_pivot_md.reset_index(inplace=True)


In [16]:
df_pivot_md=df_pivot_md.rename(columns={"Num_Acc":'Accident_Id'})

In [17]:
df = accidents_cyclistes_vehicules
df=df.sort_values(['Num_Acc','Catégorie_véhicule',"catégorie_usager"],ascending=True)
# Ajouter un index par groupe ID
df['row'] = df.groupby('Num_Acc').cumcount() + 1

# Transformer les lignes en colonnes
df_pivot_c = df.pivot(index='Num_Acc', columns='row', values=['catégorie_usager', 'gravité_blessure', 'sexe', 'année_naissance', 'motif_trajet', 'équipement_sécu1'])

# Aplatir l'index des colonnes
df_pivot_c.columns = [f"{col[0]}{col[1]}" for col in df_pivot_c.columns]

# Réinitialiser l'index
df_pivot_c.reset_index(inplace=True)

In [18]:
df_pivot_c=df_pivot_c.rename(columns={"Num_Acc":'Accident_Id'})

In [19]:
df = accidents_piétons_vehicules
df=df.sort_values(['Num_Acc','Catégorie_véhicule',"catégorie_usager"],ascending=True)
# Ajouter un index par groupe ID
df['row'] = df.groupby('Num_Acc').cumcount() + 1

# Transformer les lignes en colonnes
df_pivot_p = df.pivot(index='Num_Acc', columns='row', values=['Catégorie_véhicule','Catégorie_véhicule','catégorie_usager', 'gravité_blessure', 'sexe', 'année_naissance', 'motif_trajet', 'équipement_sécu1'])

# Aplatir l'index des colonnes
df_pivot_p.columns = [f"{col[0]}{col[1]}" for col in df_pivot_p.columns]

# Réinitialiser l'index
df_pivot_p.reset_index(inplace=True)

In [20]:
df_pivot_p=df_pivot_p.rename(columns={"Num_Acc":'Accident_Id'})

ON FUSIONNE AVEC LA TABLE DE BASE

In [21]:
accidents_mobilites_douces_pivot = df_pivot_md.merge(accidents_mobilites_douces_base,on=['Accident_Id'])
accidents_piétons_pivot = df_pivot_p.merge(accidents_piétons_base,on=['Accident_Id'])
accidents_cyclistes_pivot = df_pivot_c.merge(accidents_cyclistes_base,on=['Accident_Id'])

In [22]:
accidents_mobilites_douces_pivot.to_csv("exp/accidents_mobilite_douce_pivot_avec_carac.csv", index=False)
accidents_piétons_pivot.to_csv("exp/accidents_piétons_pivot_avec_carac.csv", index=False)
accidents_cyclistes_pivot.to_csv("exp/accidents_cyclistes_pivot_avec_carac.csv", index=False)